This will aggregate spotify data with features into daily summaries that we can use for correlations

In [4]:
import json
import pandas as pd

init_notebook_mode(connected=True)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_columns', None)


spotify_data = pd.read_csv("../data/spotifyhistory_w_features.csv")


spotify_data.head()

4.14.1


,Unnamed: 0,name,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,albumName,albumID
0,0,African Sun___Naïka,2021-01-01 10:26,Naïka,African Sun,4544,2021-01-01 10:26:00,0.802,0.4420,1,-8.908,1,0.4230,0.590,0.000023,0.107,0.4860,103.047,audio_features,7DQPyLJx2EVKlf5RTpK3Zo,spotify:track:7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/tracks/7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/audio-analysis/7DQPyLJx2EVKlf5RTpK3Zo,158486,4,"Lost in Paradïse, Pt. 1",1HgmA08XqxQ4hPrClSsLfi
1,1,I Feel Wealth and Abundance Flowing Toward Me Now___Instant Affirmations,2021-01-02 05:56,Instant Affirmations,I Feel Wealth and Abundance Flowing Toward Me Now,292197,2021-01-02 05:56:00,0.362,0.1400,0,-19.006,1,0.0467,0.797,0.014700,0.080,0.0396,87.819,audio_features,5SMNzzuRx7FDdvbo9NA3k6,spotify:track:5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/tracks/5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/audio-analysis/5SMNzzuRx7FDdvbo9NA3k6,312217,4,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej
2,2,I Easily Bring Money and Wealth Into My Life___Instant Affirmations,2021-01-02 06:01,Instant Affirmations,I Easily Bring Money and Wealth Into My Life,312217,2021-01-02 06:01:00,0.290,0.0531,0,-20.613,1,0.0316,0.717,0.060500,0.104,0.0565,98.113,audio_features,3IcjAr7J6qe6wt52r6suqa,spotify:track:3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/tracks/3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/audio-analysis/3IcjAr7J6qe6wt52r6suqa,312217,4,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej
3,3,I Am Aligned with the Energy of Wealth and Abundance___Instant Affirmations,2021-01-02 06:07,Instant Affirmations,I Am Aligned with the Energy of Wealth and Abundance,312217,2021-01-02 06:07:00,0.235,0.0829,0,-21.733,1,0.0327,0.673,0.490000,0.104,0.0371,103.446,audio_features,5FqidFOb9epSJZhKG9osPe,spotify:track:5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/tracks/5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/audio-analysis/5FqidFOb9epSJZhKG9osPe,312217,4,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej
4,4,I Am Successful Strong and Satisfied___Instant Affirmations,2021-01-02 06:11,Instant Affirmations,I Am Successful Strong and Satisfied,244553,2021-01-02 06:11:00,0.433,0.1980,7,-15.202,1,0.0331,0.823,0.207000,0.164,0.2640,139.167,audio_features,6QHDfLwLSP9NnSMFAXdE8L,spotify:track:6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/tracks/6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/audio-analysis/6QHDfLwLSP9NnSMFAXdE8L,244553,4,Affirmations for Success,00b3RaOGzalm8f58l43QaU


In [14]:
from datetime import datetime
single_date = datetime.strptime(spotify_data.iloc[0]["endTime"], '%Y-%m-%d %H:%M')

# 
# we need to get start time .. endtime - milliseconds

1609525555456.0

In [33]:
# need to convert my timezone to an ISO format that get's location
def get_startTime(endTime: str, msPlayed: int):
    date_object = datetime.strptime(endTime, '%Y-%m-%d %H:%M')
    
    # multiply & divide by 1000 because msPlayed is milliseconds and .timestamp() is in seconds
    startTimestamp = ((date_object.timestamp() *  1000) - msPlayed) / 1000 
    
    startTime = datetime.fromtimestamp(startTimestamp)
    
    return startTime.strftime('%Y-%m-%d %H:%M')

spotify_data["startTime"] = spotify_data[["endTime", "msPlayed"]].apply(lambda x: get_startTime(x['endTime'], x['msPlayed']), axis=1)

type(spotify_data.iloc[0]['startTime'])

str

In [44]:
# get date of event in y-m-d format    
spotify_data.dropna()

def get_summaryday(startTime: str):
    date_object = datetime.strptime(startTime, '%Y-%m-%d %H:%M')
    
    return date_object.strftime('%Y-%m-%d')

spotify_data["summary_day"] = spotify_data["startTime"].apply(lambda x: get_summaryday(x))


spotify_data

,Unnamed: 0,name,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,albumName,albumID,startTime,summary_day
0,0,African Sun___Naïka,2021-01-01 10:26,Naïka,African Sun,4544,2021-01-01 10:26:00,0.802,0.4420,1,-8.908,1,0.4230,0.5900,0.000023,0.1070,0.4860,103.047,audio_features,7DQPyLJx2EVKlf5RTpK3Zo,spotify:track:7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/tracks/7DQPyLJx2EVKlf5RTpK3Zo,https://api.spotify.com/v1/audio-analysis/7DQPyLJx2EVKlf5RTpK3Zo,158486,4,"Lost in Paradïse, Pt. 1",1HgmA08XqxQ4hPrClSsLfi,2021-01-01 10:25,2021-01-01
1,1,I Feel Wealth and Abundance Flowing Toward Me Now___Instant Affirmations,2021-01-02 05:56,Instant Affirmations,I Feel Wealth and Abundance Flowing Toward Me Now,292197,2021-01-02 05:56:00,0.362,0.1400,0,-19.006,1,0.0467,0.7970,0.014700,0.0800,0.0396,87.819,audio_features,5SMNzzuRx7FDdvbo9NA3k6,spotify:track:5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/tracks/5SMNzzuRx7FDdvbo9NA3k6,https://api.spotify.com/v1/audio-analysis/5SMNzzuRx7FDdvbo9NA3k6,312217,4,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:51,2021-01-02
2,2,I Easily Bring Money and Wealth Into My Life___Instant Affirmations,2021-01-02 06:01,Instant Affirmations,I Easily Bring Money and Wealth Into My Life,312217,2021-01-02 06:01:00,0.290,0.0531,0,-20.613,1,0.0316,0.7170,0.060500,0.1040,0.0565,98.113,audio_features,3IcjAr7J6qe6wt52r6suqa,spotify:track:3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/tracks/3IcjAr7J6qe6wt52r6suqa,https://api.spotify.com/v1/audio-analysis/3IcjAr7J6qe6wt52r6suqa,312217,4,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 05:55,2021-01-02
3,3,I Am Aligned with the Energy of Wealth and Abundance___Instant Affirmations,2021-01-02 06:07,Instant Affirmations,I Am Aligned with the Energy of Wealth and Abundance,312217,2021-01-02 06:07:00,0.235,0.0829,0,-21.733,1,0.0327,0.6730,0.490000,0.1040,0.0371,103.446,audio_features,5FqidFOb9epSJZhKG9osPe,spotify:track:5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/tracks/5FqidFOb9epSJZhKG9osPe,https://api.spotify.com/v1/audio-analysis/5FqidFOb9epSJZhKG9osPe,312217,4,"Affirmations for Wealth, Money and Abundance",5OaAppEqCEHtyQPUhwdFej,2021-01-02 06:01,2021-01-02
4,4,I Am Successful Strong and Satisfied___Instant Affirmations,2021-01-02 06:11,Instant Affirmations,I Am Successful Strong and Satisfied,244553,2021-01-02 06:11:00,0.433,0.1980,7,-15.202,1,0.0331,0.8230,0.207000,0.1640,0.2640,139.167,audio_features,6QHDfLwLSP9NnSMFAXdE8L,spotify:track:6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/tracks/6QHDfLwLSP9NnSMFAXdE8L,https://api.spotify.com/v1/audio-analysis/6QHDfLwLSP9NnSMFAXdE8L,244553,4,Affirmations for Success,00b3RaOGzalm8f58l43QaU,2021-01-02 06:06,2021-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,2885,Crisis And Response ___TED Radio Hour,2021-04-24 21:48,TED Radio Hour,Crisis And Response,1938250,2021-04-24 21:48:00,0.611,0.4970,0,-8.468,0,0.4970,0.8520,0.000000,0.2620,0.4280,86.414,audio_features,774eBxLGlgEA3KpTXA0WQ7,spotify:track:774eBxLGlgEA3KpTXA0WQ7,https://api.spotify.com/v1/tracks/774eBxLGlgEA3KpTXA0WQ7,https://api.spotify.com/v1/audio-analysis/774eBxLGlgEA3KpTXA0WQ7,682200,4,"Taking Liberties: Prisons, Policing and Surveillance in an Age of Crisis",5oxBDFoqU3fIyUzLHHLElG,2021-04-24 21:15,2021-04-24
2886,2886,Oshey___Juls,2021-04-24 21:52,Juls,Oshey,270158,2021-04-24 21:52:00,0.856,0.3480,8,-10.113,0,0.0784,0.0926,0.076300,0.0662,0.3700,117.983,audio_features,7086QbyiN4ITmlngO7XwgA,spotify:track:7086QbyiN4ITmlngO7XwgA,https://api.spotify.com/v1/tracks/7086QbyiN4ITmlngO7XwgA,https://api.spotify.com/v1/audio-analysis/7086QbyiN4ITmlngO7XwgA,270158,4,Ojekoo,71BxXDL7H2LPzKZ0iKUksI,2021-04-24 21:47,2021-04-24
2887,2887,"Love You Toni

In [48]:
# now let's get the daily summary for music listened to, summary here is the average of these values
spotify_data_summary = spotify_data.groupby(["summary_day"], as_index=False)

spotify_data_summary = spotify_data_summary.agg({
    'danceability': 'mean',
    "energy": 'mean',
    "key": "mean",
    "loudness": "mean",
    "mode": "mean",
    "speechiness": "mean",
    "acousticness": "mean",
    "instrumentalness": "mean",
    "liveness": "mean",
    "valence": "mean",
    "tempo": "mean"
})

spotify_data_summary

,summary_day,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2021-01-01,0.802000,0.442000,1.000000,-8.908000,1.000000,0.423000,0.590000,0.000023,0.107000,0.486000,103.047000
1,2021-01-02,0.377395,0.163132,4.894737,-19.832263,0.657895,0.094379,0.768918,0.191052,0.130745,0.151003,105.038447
2,2021-01-03,0.679000,0.573875,5.875000,-7.405875,0.625000,0.065412,0.426250,0.032080,0.135150,0.475425,109.305625
3,2021-01-04,0.604000,0.568714,4.142857,-8.329714,0.571429,0.105029,0.355714,0.100036,0.144371,0.294886,110.243571
4,2021-01-05,0.761350,0.708650,5.900000,-5.945000,0.450000,0.150110,0.330305,0.002027,0.144360,0.574600,114.481700
...,...,...,...,...,...,...,...,...,...,...,...,...
98,2021-04-19,0.682190,0.634048,4.142857,-6.839619,0.380952,0.168776,0.249800,0.049855,0.147024,0.531286,111.388000
99,2021-04-20,0.686803,0.547702,6.049180,-8.110934,0.426230,0.103607,0.357877,0.281365,0.156693,0.454449,116.559115
100,2021-04-22,0.636729,0.495953,5.411765,-9.972271,0.552941,0.145389,0.420362,0.254852,0.157574,0.425169,111.160741
101,2021-04-23,0.649747,0.350723,5.346667,-12.886187,0.560000,0.079697,0.574325,0.653720,0.139753,0.341071,107.470960


In [49]:
spotify_data_summary.to_csv("../data/spotify_data_summary.csv", index=False)